In [1]:
import numpy as np
from bokeh.io import show,output_notebook,output_file
from bokeh.models import Button, DataTable, TableColumn, PointDrawTool, PolyDrawTool, PolyEditTool, ColumnDataSource,TextInput
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LassoSelectTool,
    BoxSelectTool,
    LogColorMapper,
    CustomJS,
    Slider,
    CheckboxGroup,
    Select,
    Div,
    TapTool,
    ColorBar,
    LinearColorMapper,
    CategoricalColorMapper,
    DataTable,
    TableColumn,
    NumberFormatter,
    NumeralTickFormatter
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.layouts import (widgetbox, row, column, gridplot, layout)

from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TONER_BACKGROUND,STAMEN_TONER,STAMEN_TERRAIN

from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata import us_states
from bokeh import events

import pandas as pd
from bokeh.colors.groups import yellow

import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point, Polygon, MultiPolygon,MultiLineString

from TEN import TEN
TX = TEN('plot')


Loading sub
Loading genbus
Loading bus2sub
Loading windfarm
Loading branches
Loading NRELsites
Done loading


In [2]:
us_states = us_states.data.copy()

del us_states["HI"]
del us_states["AK"]

#states_xys = [reproject_wgs_to_itm([us_states[code]["lons"],us_states[code]["lats"]]) for code in us_states]

state_xs = [us_states[code]["lons"] for code in us_states]
state_ys = [us_states[code]["lats"] for code in us_states]

In [3]:
tx_counties = {
    code: county for code, county in counties.items() if county["state"] == "tx"
}

tx_county_xs = [county["lons"] for county in tx_counties.values()]
tx_county_ys = [county["lats"] for county in tx_counties.values()]

tx_county_names = [county['name'] for county in tx_counties.values()]

In [4]:
branch = pd.read_csv('branchTest')

In [5]:
cities = branch['withinCityStart'].unique().tolist()[1:]
TX_name_list = [{'city':s,'state':'Texas'} for s in cities]
gdf = ox.gdf_from_places(TX_name_list)

In [6]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    if row[geom].type == 'MultiPolygon':
        exterior = row[geom].geoms[0].exterior
    elif row[geom].type == 'Point' or row[geom].type == 'LineString':
        return
    else:
        exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [7]:
gdf['x'] = gdf.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
gdf['y'] = gdf.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

In [8]:
gdf = gdf[(gdf['geometry'].type == 'Polygon') | (gdf['geometry'].type == 'MultiPolygon')]

In [9]:
p = figure(
    title="Test",toolbar_location="left",plot_width=1200, plot_height=700,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None
p.toolbar.logo = None
# US State boarder
p.patches(state_xs,state_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=1.5)

# City Data
city_data = {'x': gdf['x'].values.tolist(), 'y': gdf['y'].values.tolist(),'name':gdf['place_name'].values.tolist()}
source = ColumnDataSource(city_data)
city_patch =p.patches(source=source,xs='x', ys='y', fill_alpha=0.2,
    line_color="red", line_alpha=0.5, line_width=1.5)
hover2 = HoverTool(renderers=[city_patch])
hover2.point_policy = "follow_mouse"
hover2.tooltips = [
    ("name", "@name")
]
p.add_tools(hover2)

data_branch = {'x': branch[['from_lon','to_lon']].values.tolist(), 'y': branch[['from_lat','to_lat']].values.tolist(),
                        'cityStart':branch['withinCityStart'].values.tolist(),
                        'cityEnd':branch['withinCityEnd'].values.tolist(),
                        'countyStart':branch['withinCountyStart'].values.tolist(),
                        'countyEnd':branch['withinCountyEnd'].values.tolist()}
s_branch = ColumnDataSource(data_branch)
lines = p.multi_line(source = s_branch ,xs='x',ys='y',line_alpha=0.3,line_color='black')
hover1 = HoverTool(renderers=[lines])
hover1.point_policy = "follow_mouse"
hover1.tooltips = [
    ("cityStart", "@cityStart"),
    ("cityStart", "@cityStart"),
    ("countyStart", "@countyStart"),
    ("countyEnd", "@countyEnd"),
]
p.add_tools(hover1)
p.patches(tx_county_xs,tx_county_ys, fill_alpha=0.2,
    line_color="gray", line_alpha=0.1, line_width=1.5)

GlyphRenderer(id='e26dabde-363b-4848-8e53-24bc92247d07', ...)

In [14]:
show(p)

### Analyse Congestion

In [11]:
branch = pd.read_csv('branchTest')

In [12]:
TX.CongStatBase = pd.concat([TX.CongStatBase, branch['dist'],branch['baseKV'],
                             branch['withinCityStart'],branch['withinCityEnd'],
                             branch['withinCountyStart'],branch['withinCountyEnd'],
                             branch['withinZoneStart'],branch['withinZoneEnd']],axis=1)

In [13]:
TX.CongStatBase.sort_values('hutil>=0p9',ascending=False)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist,baseKV,withinCityStart,withinCityEnd,withinCountyStart,withinCountyEnd,withinZoneStart,withinZoneEnd
line,,,,,,,,,,,,,,
697,40,0,18101,39,18101,18140,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
696,40,0,18092,56,18092,18148,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
699,40,0,18087,46,18087,18133,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
698,40,0,18084,65,18084,18149,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
700,40,0,18067,45,18067,18112,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
695,40,0,18047,60,18047,18107,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
556,238,12504,4381,931,16885,17816,0.000,13.8,none,none,Goliad,Goliad,South,South
555,238,12504,4381,931,16885,17816,0.000,13.8,none,none,Goliad,Goliad,South,South
557,238,12504,4381,931,16885,17816,0.000,13.8,none,none,Goliad,Goliad,South,South


### City

In [27]:
WithinCity = TX.CongStatBase.loc[TX.CongStatBase['withinCityStart'] == TX.CongStatBase['withinCityEnd']]
WithinCity_group = WithinCity.groupby('withinCityStart')
# Sum of quantities in each city
WithinCity_group.sum().sort_values('hutil>=0p9',ascending=False)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist,baseKV
withinCityStart,,,,,,,,
none,801043,158648,413287,394388,571935,966323,34030.804,258501.6
Wichita Falls,4624,0,9625,181,9625,9806,18.038,1002.0
San Antonio,60133,0,36,116,36,152,643.203,14209.8
Dallas,41247,0,6,70,6,76,364.205,11129.0
Waco,251,0,0,0,0,0,4.920,161.0
Round Rock,4306,0,0,6,0,6,21.820,1347.8
Mesquite,1604,0,0,0,0,0,22.631,483.0
New Braunfels,2789,0,0,0,0,0,0.000,460.0
North Richland Hills,497,0,0,0,0,0,10.830,322.0


### County

In [28]:
WithinCounty = TX.CongStatBase.loc[TX.CongStatBase['withinCountyStart'] == TX.CongStatBase['withinCountyEnd']]
WithinCounty_group = WithinCounty.groupby('withinCountyStart')
# Sum of quantities in each county
WithinCounty_group.sum().sort_values('hutil>=0p9',ascending=False)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist,baseKV
withinCountyStart,,,,,,,,
Hidalgo,14953,0,109306,21653,109306,130959,523.071,6176.2
Goliad,3172,37512,13143,2793,50655,53448,74.627,1191.4
Sterling,1903,20870,19575,8194,40445,48639,52.164,559.6
Nolan,7192,12275,27825,28479,40100,68579,183.976,2945.8
Kenedy,4091,8186,25321,8259,33507,41766,163.889,1709.4
Scurry,3365,13097,18166,12583,31263,43846,117.170,1877.4
Howard,1199,1119,29331,11496,30450,41946,31.719,542.8
Taylor,6019,14119,15466,10979,29585,40564,321.106,2604.2
Martin,500,1824,14040,4650,15864,20514,50.919,248.0


### Zone

In [29]:
WithinZone = TX.CongStatBase.loc[TX.CongStatBase['withinZoneStart'] == TX.CongStatBase['withinZoneEnd']]
WithinZone_group = WithinZone.groupby('withinZoneStart')
# Sum of quantities in each Zone
WithinZone_group.sum().sort_values('hutil>=0p9',ascending=False)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist,baseKV
withinZoneStart,,,,,,,,
South,79579,59145,177142,73964,236287,310251,5443.615,34569.0
West,36374,80523,118873,97253,199396,296649,3446.285,19071.0
Far West,19311,12306,82511,51290,94817,146107,2779.249,10306.8
North,40965,1674,15230,30131,16904,47035,2123.463,16316.6
Coast,351887,1147,8965,66272,10112,76384,7302.362,105688.0
none,8326,2144,7321,5818,9465,15283,480.155,3190.4
North Central,421089,1259,7995,58210,9254,67464,9152.651,129826.2
East,74432,996,5164,3948,6160,10108,3129.040,24905.0
South Central,249496,121,3016,8707,3137,11844,6032.133,77382.4


### Distance

In [35]:
TX.CongStatBase.sort_values(['dist','hutil>=0p9'],ascending=True)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist,baseKV,withinCityStart,withinCityEnd,withinCountyStart,withinCountyEnd,withinZoneStart,withinZoneEnd
line,,,,,,,,,,,,,,
6,250,0,0,0,0,0,0.000,230.0,none,none,Borden,Borden,Far West,Far West
16,120,0,0,36,0,36,0.000,13.8,none,none,Pecos,Pecos,Far West,Far West
19,46,0,0,0,0,0,0.000,22.0,none,none,Presidio,Presidio,Far West,Far West
81,124,0,0,0,0,0,0.000,18.0,none,none,Ward,Ward,Far West,Far West
82,117,0,0,0,0,0,0.000,18.0,none,none,Ward,Ward,Far West,Far West
83,110,0,0,0,0,0,0.000,18.0,none,none,Ward,Ward,Far West,Far West
84,250,0,0,0,0,0,0.000,230.0,none,none,Ward,Ward,Far West,Far West
93,33,0,0,0,0,0,0.000,22.0,none,none,Pecos,Pecos,Far West,Far West
94,27,0,0,0,0,0,0.000,22.0,none,none,Pecos,Pecos,Far West,Far West


In [43]:
TX.CongStatBase.groupby('dist').mean()

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,baseKV
dist,,,,,,,
0.000,589.769767,159.598837,391.154651,229.477907,550.753488,780.231395,78.309767
1.242,250.000000,0.000000,0.000000,0.000000,0.000000,0.000000,115.000000
1.704,123.500000,0.000000,0.000000,0.000000,0.000000,0.000000,115.000000
1.763,180.000000,0.000000,0.000000,0.000000,0.000000,0.000000,115.000000
1.847,251.000000,0.000000,0.000000,0.000000,0.000000,0.000000,161.000000
1.927,409.000000,0.000000,0.000000,0.000000,0.000000,0.000000,161.000000
2.041,297.000000,0.000000,0.000000,0.000000,0.000000,0.000000,115.000000
2.074,298.000000,0.000000,0.000000,0.000000,0.000000,0.000000,161.000000
2.250,293.750000,0.000000,0.000000,0.000000,0.000000,0.000000,115.000000


In [45]:
TX.CongStatBase.loc[TX.CongStatBase['dist'] < 1.].sort_values('hutil>=0p9',ascending=False)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist,baseKV,withinCityStart,withinCityEnd,withinCountyStart,withinCountyEnd,withinZoneStart,withinZoneEnd
line,,,,,,,,,,,,,,
697,40,0,18101,39,18101,18140,0.0,13.8,none,none,Hidalgo,Hidalgo,South,South
696,40,0,18092,56,18092,18148,0.0,13.8,none,none,Hidalgo,Hidalgo,South,South
699,40,0,18087,46,18087,18133,0.0,13.8,none,none,Hidalgo,Hidalgo,South,South
698,40,0,18084,65,18084,18149,0.0,13.8,none,none,Hidalgo,Hidalgo,South,South
700,40,0,18067,45,18067,18112,0.0,13.8,none,none,Hidalgo,Hidalgo,South,South
695,40,0,18047,60,18047,18107,0.0,13.8,none,none,Hidalgo,Hidalgo,South,South
557,238,12504,4381,931,16885,17816,0.0,13.8,none,none,Goliad,Goliad,South,South
555,238,12504,4381,931,16885,17816,0.0,13.8,none,none,Goliad,Goliad,South,South
556,238,12504,4381,931,16885,17816,0.0,13.8,none,none,Goliad,Goliad,South,South


### Voltage

In [31]:
TX.CongStatBase.sort_values(['baseKV','hutil>=0p9'],ascending=False)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist,baseKV,withinCityStart,withinCityEnd,withinCountyStart,withinCountyEnd,withinZoneStart,withinZoneEnd
line,,,,,,,,,,,,,,
2354,2295,0,6598,3494,6598,10092,49.475,500.0,none,none,Calhoun,Matagorda,Coast,Coast
2450,2020,909,1497,1185,2406,3591,54.832,500.0,none,none,Wharton,Matagorda,Coast,Coast
113,2000,0,0,0,0,0,0.000,500.0,none,none,Ector,Ector,Far West,Far West
116,2300,0,0,0,0,0,177.174,500.0,none,none,Nolan,Ector,West,Far West
117,2300,0,0,0,0,0,177.174,500.0,none,none,Nolan,Ector,West,Far West
131,2295,0,0,0,0,0,75.344,500.0,none,none,Lamar,Fannin,North,North
132,2020,0,0,0,0,0,23.727,500.0,none,none,Fannin,Grayson,North,North
147,2020,0,0,0,0,0,83.836,500.0,none,none,Hemphill,Carson,none,none
159,2020,0,0,0,0,0,196.813,500.0,none,none,Carson,Crosby,none,North


In [32]:
TX.CongStatBase.groupby('baseKV').mean()

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist
baseKV,,,,,,,
13.2,1085.619048,18.142857,27.904762,43.523810,46.047619,89.571429,0.000000
13.8,625.323944,279.626761,779.746479,461.415493,1059.373239,1520.788732,0.000000
18.0,417.680412,335.185567,738.247423,284.927835,1073.432990,1358.360825,0.000000
20.0,309.476190,70.714286,156.238095,129.619048,226.952381,356.571429,0.000000
22.0,708.142857,272.071429,491.428571,411.071429,763.500000,1174.571429,0.000000
24.0,456.650000,181.700000,247.500000,144.550000,429.200000,573.750000,0.000000
115.0,244.574550,13.738432,51.428021,73.942159,65.166452,139.108612,13.613666
161.0,454.856757,0.444595,10.001351,121.643243,10.445946,132.089189,13.178468
230.0,629.424658,52.976027,75.434932,66.917808,128.410959,195.328767,27.682397


### Capacity

In [33]:
TX.CongStatBase.sort_values(['hutil>=0p9','Capacity'],ascending=False)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist,baseKV,withinCityStart,withinCityEnd,withinCountyStart,withinCountyEnd,withinZoneStart,withinZoneEnd
line,,,,,,,,,,,,,,
697,40,0,18101,39,18101,18140,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
696,40,0,18092,56,18092,18148,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
699,40,0,18087,46,18087,18133,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
698,40,0,18084,65,18084,18149,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
700,40,0,18067,45,18067,18112,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
695,40,0,18047,60,18047,18107,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
555,238,12504,4381,931,16885,17816,0.000,13.8,none,none,Goliad,Goliad,South,South
556,238,12504,4381,931,16885,17816,0.000,13.8,none,none,Goliad,Goliad,South,South
557,238,12504,4381,931,16885,17816,0.000,13.8,none,none,Goliad,Goliad,South,South


In [34]:
TX.CongStatBase.groupby('Capacity').max()

,hutil1,hutil0p9-1,hutil0p8-0p9,hutil>=0p9,hutil>=0p8,dist,baseKV,withinCityStart,withinCityEnd,withinCountyStart,withinCountyEnd,withinZoneStart,withinZoneEnd
Capacity,,,,,,,,,,,,,
17,0,0,0,0,0,0.000,13.8,none,none,Coke,Coke,West,West
19,0,0,0,0,0,0.000,115.0,none,none,Kent,Kent,North,North
21,0,0,0,0,0,0.000,13.8,none,none,Shackelford,Shackelford,North Central,North Central
24,0,0,0,0,0,50.758,500.0,none,none,Haskell,Mills,North Central,North Central
25,0,0,0,0,0,0.000,13.8,none,none,Williamson,Williamson,South Central,South Central
27,0,0,0,0,0,0.000,22.0,none,none,Pecos,Pecos,Far West,Far West
28,0,0,0,0,0,0.000,13.8,none,none,Hidalgo,Hidalgo,South,South
29,0,0,0,0,0,41.899,230.0,none,none,Wilson,Karnes,South Central,South Central
30,0,0,0,0,0,39.865,230.0,none,none,DeWitt,Fayette,South Central,South Central
